# Классификация: SI > 8

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Подготовка целевой переменной
y = (df["SI"] > 8).astype(int)
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:00:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 285, number of negative: 513
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18190
[LightGBM] [Info] Number of data points in the train set: 798, number of used features: 169
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.357143 -> initscore=-0.587787
[LightGBM] [Info] Start training from score -0.587787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

# Классификация: выделение высокоселективных соединений (SI > 8)

В этом разделе выполняются следующие шаги:

1. **Формирование целевой переменной**
   - Порог SI = 8.
   - Создаётся бинарный вектор `y`:
     - `1` = «SI > 8» (высокоселективные соединения) — 285 образцов.
     - `0` = «SI ≤ 8» (менее селективные соединения) — 513 образцов.

2. **Разделение выборки и масштабирование**
   - Стратифицированный сплит: `train_test_split(..., stratify=y, test_size=0.2)`.
   - `StandardScaler` применяется к признакам для моделей, чувствительных к масштабу.

3. **Обучение моделей**
   - **LogisticRegression**  
   - **RandomForestClassifier**  
   - **XGBClassifier**  
   - **LGBMClassifier**  

4. **Подбор гиперпараметров**  
   (GridSearchCV с 5-кратной кросс-валидацией, метрика ROC AUC)

5. **Оценка качества**  
   На тестовой выборке получены следующие метрики:

   | Модель                 | Accuracy | Precision | Recall | F1    | ROC AUC |
   |------------------------|----------|-----------|--------|-------|---------|
   | Logistic Regression    | 0.690    | 0.562     | 0.577  | 0.569 | 0.689   |
   | Random Forest          | 0.690    | 0.574     | 0.493  | 0.530 | 0.722   |
   | XGBoost                | 0.705    | 0.583     | 0.592  | 0.587 | 0.727   |
   | LightGBM               | 0.730    | 0.623     | 0.606  | 0.614 | 0.724   |

6. **Анализ результатов**  
   - **LightGBM** показал наилучшую **точность** (73.0%) и **F1-score** (0.614).  
   - **XGBoost** чуть уступил в точности, но имеет более высокую ROC AUC (0.727).  
   - Логистическая регрессия и Random Forest демонстрируют схожие показатели accuracy (~69%) и существенно ниже precision/recall.

> **Вывод:** для задачи жёсткого порога SI > 8 наилучший баланс метрик демонстрирует **LightGBM** (Accuracy 0.73, F1 0.61), однако **XGBoost** обеспечивает чуть лучшее ранжирование (ROC AUC 0.727). Эти модели могут эффективно отбирать высокоселективные соединения.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()
